<a href="https://colab.research.google.com/github/Rifat429/DL/blob/main/RNN_QA_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [1]:
import pandas as pd

df = pd.read_csv("/content/100_Unique_QA_Dataset.csv")

In [2]:
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [3]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [4]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [5]:
# vocab
vocab = {'<UNK>':0}

In [6]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)


In [7]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [8]:
len(vocab)

324

In [9]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [10]:
text_to_indices("What is campusx", vocab)

[1, 2, 0]

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader

In [12]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [13]:
dataset = QADataset(df, vocab)

In [14]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [15]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[ 42, 299, 300, 118,  14, 301, 302, 158, 303, 304, 305, 306]]) tensor([307])
tensor([[  1,   2,   3, 212,   5,  14, 213, 214]]) tensor([215])
tensor([[ 10, 140,   3, 141, 270,  93, 271,   5,   3, 272]]) tensor([273])
tensor([[ 42, 137,   2,  62,  39,   3, 322, 323]]) tensor([6])
tensor([[  1,  87, 229, 230, 231, 232]]) tensor([233])
tensor([[ 1,  2,  3, 24, 25,  5, 26, 19, 27]]) tensor([28])
tensor([[  1,   2,   3, 146,  86,  19, 192, 193]]) tensor([194])
tensor([[  1,   2,   3, 221,   5, 222, 223, 224]]) tensor([225])
tensor([[ 1,  2,  3, 37, 38, 39, 40]]) tensor([41])
tensor([[ 42, 263, 264,  14, 265, 266, 158, 267]]) tensor([268])
tensor([[ 10,  75,   3, 296,  19, 297]]) tensor([298])
tensor([[ 78,  79, 288,  81,  19,  14, 289]]) tensor([85])
tensor([[ 10,  11, 189, 158, 190]]) tensor([191])
tensor([[  1,   2,   3, 122, 123,  19,   3,  45]]) tensor([124])
tensor([[ 10,  96,   3, 104, 239]]) tensor([240])
tensor([[ 42, 137, 118,   3, 247,   5, 248]]) tensor([249])
tensor([[ 7

In [16]:
import torch.nn as nn

In [17]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [18]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [26]:
learning_rate = 0.01
epochs = 200

In [27]:
model = SimpleRNN(len(vocab))

model = model.to(device)

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [29]:

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:
    # move data to gpu
    question, answer = question.to(device), answer.to(device)


    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 537.680854
Epoch: 2, Loss: 341.908591
Epoch: 3, Loss: 164.393935
Epoch: 4, Loss: 76.749801
Epoch: 5, Loss: 39.299294
Epoch: 6, Loss: 24.981628
Epoch: 7, Loss: 38.415214
Epoch: 8, Loss: 31.770621
Epoch: 9, Loss: 29.313525
Epoch: 10, Loss: 21.803498
Epoch: 11, Loss: 20.882335
Epoch: 12, Loss: 18.685954
Epoch: 13, Loss: 9.138175
Epoch: 14, Loss: 9.134043
Epoch: 15, Loss: 5.394810
Epoch: 16, Loss: 10.919091
Epoch: 17, Loss: 9.157739
Epoch: 18, Loss: 10.880218
Epoch: 19, Loss: 6.245164
Epoch: 20, Loss: 2.114064
Epoch: 21, Loss: 6.841804
Epoch: 22, Loss: 2.006237
Epoch: 23, Loss: 0.972741
Epoch: 24, Loss: 0.368183
Epoch: 25, Loss: 0.302751
Epoch: 26, Loss: 0.266139
Epoch: 27, Loss: 0.238190
Epoch: 28, Loss: 0.216716
Epoch: 29, Loss: 0.198774
Epoch: 30, Loss: 0.184057
Epoch: 31, Loss: 0.172114
Epoch: 32, Loss: 0.159765
Epoch: 33, Loss: 0.149608
Epoch: 34, Loss: 0.140370
Epoch: 35, Loss: 0.131843
Epoch: 36, Loss: 0.123881
Epoch: 37, Loss: 0.116695
Epoch: 38, Loss: 0.110219
Epoc

In [32]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

    # Move the question_tensor to the same device as the model
  question_tensor = question_tensor.to(device) # Move to device

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [33]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [34]:
list(vocab.keys())[7]

'paris'